In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [2]:
'''
Loading chargers from JSON
'''
deep_reload(src)

vertices=src.store.Load('vertices.json')

In [3]:
'''
Making a graph from the vertices
'''
deep_reload(src)

graph=src.network.NX_Graph_From_Vertices(
    vertices,
)

In [4]:
'''
Nodes that might feasibly be included - roundtrip to depot is less than max range
'''
max_route_distance=500e3
max_route_time=5*3600

feasible_nodes=[]

depot='1000000000'

for destination in graph.nodes:
    rt_distance=graph[depot][destination]['length']+graph[destination][depot]['length']
    rt_time=graph[depot][destination]['time']+graph[destination][depot]['time']

    if (rt_distance<=max_route_distance)&(rt_time<=max_route_time):
        feasible_nodes.append(destination)
    # break
len(feasible_nodes)
# nodes=[node for node,info in graph._node.items() if node

872

In [5]:
feasible_nodes.remove(depot)

feasible_nodes=np.random.choice(feasible_nodes,200,replace=False).tolist()
feasible_nodes.append(depot)

In [6]:
import networkx as nx

distance_adjacency=nx.to_numpy_array(graph,nodelist=feasible_nodes,weight='length')
time_adjacency=nx.to_numpy_array(graph,nodelist=feasible_nodes,weight='time')
# adj.shape

In [7]:
'''
Making a graph from the vertices
'''
deep_reload(src)

node_to_idx,idx_to_node=src.router.Assignments(feasible_nodes)

In [17]:
'''
Computing routes
'''
deep_reload(src)

depot_indices=[node_to_idx['1000000000']]

node_time=1800

for n in graph._node.values():
    n['time']=node_time


routes,success=src.router.ClarkeWright(
    distance_adjacency,
	time_adjacency,
	depot_indices,
	max_iterations=int(1e3),
	mode='distance', # ['distance','time']
	max_route_distance=500e3,
	max_leg_distance=500e3,
	min_leg_distance=20e3,
    distance_offset=0,
	max_route_time=5*3600,
	max_leg_time=5*3600,
	min_leg_time=0,
    time_offset=1800,
)

print(f'\n{success}')

[999, 4317036820.0, 3.4168481826782224e-05]
False


In [18]:
np.array([len(route) for route in routes])

array([3, 3, 3, 3, 3, 3, 5, 3, 3, 6, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 5, 3, 3, 5, 3, 3, 3, 5, 3, 3, 3, 3, 3, 5, 3, 3,
       3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [19]:
deep_reload(src)

routes_nodes=[[idx_to_node[stop] for stop in route] for route in routes]
routes_nodes_opt=([
    src.router.RouteOptimization(graph,route) for route in routes_nodes])

In [20]:
routes_nodes_opt

[['1000000000', '175318', '1000000000'],
 ['1000000000', '251768', '1000000000'],
 ['1000000000', '206782', '1000000000'],
 ['1000000000', '236837', '1000000000'],
 ['1000000000', '212958', '1000000000'],
 ['1000000000', '228229', '1000000000'],
 ['1000000000', '164062', '236852', '181598', '1000000000'],
 ['1000000000', '196166', '1000000000'],
 ['1000000000', '51236', '1000000000'],
 ['1000000000', '176651', '178608', '170936', '182614', '1000000000'],
 ['1000000000', '178028', '1000000000'],
 ['1000000000', '259506', '1000000000'],
 ['1000000000', '196378', '1000000000'],
 ['1000000000', '175203', '1000000000'],
 ['1000000000', '204655', '1000000000'],
 ['1000000000', '158989', '1000000000'],
 ['1000000000', '39930', '196048', '226533', '1000000000'],
 ['1000000000', '250841', '1000000000'],
 ['1000000000', '148514', '1000000000'],
 ['1000000000', '100486', '1000000000'],
 ['1000000000', '181350', '1000000000'],
 ['1000000000', '228301', '1000000000'],
 ['1000000000', '230915', '100